In [1]:
import numpy as np

In [2]:
x = np.arange(0,10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
import numpy as np

x = np.arange(0, 144)
seq_len = 288
slider = 4

for i in range(0, len(x) - seq_len + 1, slider):
    inputs = x[i:i + seq_len]
    targets = x[i + seq_len - 1]
    print(f'inputs: {inputs}, targets: {targets}')

